<a href="https://colab.research.google.com/github/prateeknigam9/Meme-classifier/blob/main/Deployement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# %cd '/content/drive/MyDrive/Colab Notebooks/Case_study2/data'
!mkdir /content/test_cases/

Mounted at /content/drive


In [ ]:
!curl -LkO https://raw.githubusercontent.com/remoteit/installer/master/scripts/auto-install.sh
! chmod +x ./auto-install.sh
! sudo ./auto-install.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12956  100 12956    0     0  67832      0 --:--:-- --:--:-- --:--:-- 67832

 curl found 
********************************************************
remote.it platform and binary tester version 2.5.38 
Current directory /content
Tue Apr 13 13:41:33 UTC 2021
********************************************************
Linux e122a3759df9 4.19.112+ #1 SMP Thu Jul 23 08:00:38 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux
Detected architecture is x86_64
/usr/bin/dpkg
Debian OS detected.
amd64 architecture detected.

Looks compatible.
connectd.x86_64-ubuntu16.04 is compatible!
filename connectd_2.5.38_amd64.deb
filepath https://github.com/remoteit/installer/releases/download/v2.5.38/connectd_2.5.38_amd64.deb
Selecting previously unselected package connectd.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unp

In [ ]:
! sudo connectd_installer

/usr/bin/connectd_installer: 67: /usr/bin/connectd_options: ip: not found

remote.it connection installer Version: 2.5.38 lib_v2.1.26 
Modified: June 10, 2020 (library) November 10, 2020 
Build date: Thu Nov 12 17:55:43 UTC 2020
curl with SSL support installed.

Checking the daemon for compatibility... 

Using architecture x86_64-ubuntu16.04...

Checking your network for compatibility... 

Network connection OK...
Your network is compatible with remote.it services.

********************* Sign In Menu *********************
                                                      
     1) Sign in to your remote.it account    
     2) Exit                                          
                                                      
********************************************************

Choose a menu selection (1 - 2):
1

Please enter your remote.it Username (e-mail address): 
prateeknigam9@gmail.com

Please enter your remote.it password: 
.
/usr/bin/connectd_installer: 238: /usr/bin/co

In [ ]:
!pip install easyocr

     |████████████████████████████████| 63.0MB 44kB/s 


In [ ]:
!pip install streamlit

     |████████████████████████████████| 8.2MB 6.7MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 4.2MB 57.0MB/s 
     |████████████████████████████████| 112kB 60.0MB/s 
     |████████████████████████████████| 163kB 55.7MB/s 
     |████████████████████████████████| 122kB 54.7MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=3bda16243626b03e3c912081ec128dbeef7f5b760b811bc249c574e2d42b3240
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [ ]:
# ! ls '/content/drive/My Drive/Colab Notebooks/Case_study2/data/'
# model = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Case_study2/data/Finalmodel_2.h5")

In [ ]:
%%writefile app.py

import numpy as np
import pandas as pd
import easyocr
import cv2
import re
import warnings
warnings.filterwarnings("ignore")
from keras.preprocessing import image
from io import BytesIO

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer,tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import io
import json
from tensorflow import keras

import streamlit as st
from PIL import Image



def clean_text(line):
    line = re.sub('<[^>]*>',' ',line).strip()
    line = line.strip().replace('\n','')
    line = re.sub('[^A-Za-z0-9]+',' ',line).strip()
    line = re.sub("([\(\[]).*?([\)\]])", "", line)
    line = re.sub('\W+', " ", line)
    line = re.sub("[0-9]",' ',line)
    return line.lower().strip()

def text_from_img(img_path):
  try:
    output = reader.readtext(img_path)
    sentence_=[]
    for desc in output:
      sentence_.append(desc[1])
      sent = ' '.join(sentence_)
  except:
    sent=''
  sentence = clean_text(sent)
  return sentence

def imtoarray(img_path,tsize=(256,256,3)):
  image_ = []
  try:
    im = Image.open(img_path)
    im.save(os.path.join('/','content','test_cases',str(img_path.name)))
    # rgb_im = im.convert('RGB')
    img = image.load_img(os.path.join('/','content','test_cases',img_path.name), target_size=tsize, grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    image_.append(img)
  except Exception as e:
    st.write("error: ",e)
    raise Exception
  return np.array(image_)

def process_text():
  sequence = tokenizer.texts_to_sequences([text_from_img(img_path),])
  padded = pad_sequences(sequence,maxlen=25,padding='post')
  return padded


st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Meme Classifier")
st.text("will it be popular or not")
st.text("")
st.text("")
with st.spinner('loading model into memory....'):
  @st.cache(allow_output_mutation=True)
  def load_models(model_name = '/content/drive/My Drive/Colab Notebooks/Case_study2/data/tflitemodel_2.tflite'):
      with open('/content/drive/My Drive/Colab Notebooks/Case_study2/data/tokenizer.json') as f:
          data = json.load(f)
      tokenizer = tokenizer_from_json(data)
      interpreter = tf.lite.Interpreter(model_path=model_name)
      model = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Case_study2/data/Finalmodel_2.h5")
      return tokenizer, interpreter, model

  with st.spinner('loading model into memory....'):
      tokenizer, interpreter, model = load_models()


img_path = st.file_uploader("Upload an image", type=["jpg","png","jpeg"])

if img_path is not None:
  # img_path = img_path.name
  with st.spinner("Preprocessing...."):
      text = process_text()    
      # text_data = text.astype('float32')
      img_data = imtoarray(img_path)

  st.write("predicting class: ")

  with st.spinner("Classifying...."):
      # input_details = interpreter.get_input_details()
      # output_details = interpreter.get_output_details()
      # interpreter.allocate_tensors()
      # interpreter.set_tensor(input_details[0]['index'],tf.reshape(img_data,[1,256,256,3]))
      # interpreter.set_tensor(input_details[1]['index'],tf.reshape(text_data,[1,25]))
      # interpreter.invoke()
      # output_data = interpreter.get_tensor(output_details[0]['index'])
      # st.write("#########output: ", output_details)
      test_ = text,img_data
      pred = model.predict(test_)
      pred = (pred[0][1]).astype('float32')
      if pred > 0.5:
        pred_val = "{:.2f}".format(pred*100)
        st.write(f"Given Meme might go Popular with a confidence score of {pred_val}%")
      else:
        pred_val = "{:.2f}".format(pred*100)
        st.write(f"Might not go Popular as confidence score {pred_val}% is less than 50%")
      st.write("")
      byteImgIO = io.BytesIO()
      byteImg = Image.open(img_path)
      # im = Image.open(BytesIO(img_data))
      st.image(byteImg, caption='Classifing meme',use_column_width=True)

Overwriting app.py


In [ ]:
!streamlit run --server.port 80 app.py --&>/dev/null&